In [1]:
versioninfo()

Julia Version 1.5.3
Commit 788b2c77c1 (2020-11-09 13:37 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)
Environment:
  JULIA_NUM_THREADS = 4


## BitonicSorter

ref: 

+ [Bitonic sorter : Wikipedia(en)](https://en.wikipedia.org/wiki/Bitonic_sorter)
+ [バイトニックソート : Wikipedia(ja)](https://ja.wikipedia.org/wiki/%E3%83%90%E3%82%A4%E3%83%88%E3%83%8B%E3%83%83%E3%82%AF%E3%82%BD%E3%83%BC%E3%83%88)

In [2]:
module BitonicSorter

using Base.Sort: Algorithm
using Base.Order: Ordering, lt

export BitonicSort

struct BitonicSortAlg <: Algorithm end
const BitonicSort = BitonicSortAlg()

function Base.sort!(x::AbstractVector, lo::Integer, hi::Integer, ::BitonicSortAlg, o::Ordering)
    lo ≥ hi && return x

    fullsize::Int = hi - lo
    d = sizeof(Int) * 8 - leading_zeros(fullsize - 1)  # == ceil(Int, log(2, fullsize))

    for p = 1:d, q = 1:p
        _sort_kernel!(x, lo, hi, p, q, o)
    end
    return x
end

function _sort_kernel!(x::AbstractVector, lo, hi, p, q, o)
    # @assert p ≥ q
    fullsize_1 = hi - lo
    d = 1 << UInt(p - q)
    for s = 0:2d:fullsize_1, ioff = 0:d-1
        joff = q == 1 ? (2d - ioff - 1) : ioff + d
        i = lo + s + ioff
        j = lo + s + joff
        if !lt(o, x[i], x[j])
            x[i], x[j] = x[j], x[i]
        end
    end
end

end # module

Main.BitonicSorter

In [3]:
module MTBitonicSorter

using Base.Sort: Algorithm
using Base.Order: Ordering, lt
using Base.Threads

export MTBitonicSort

struct MTBitonicSortAlg <: Algorithm end
const MTBitonicSort = MTBitonicSortAlg()
const PARALLEL_THRESHOLD = 4096
const PARALLEL_THRESHOLD_D = 12  # == log(2, PARALLEL_THRESHOLD)

function Base.sort!(x::AbstractVector, lo::Integer, hi::Integer, ::MTBitonicSortAlg, o::Ordering)
    lo ≥ hi && return x

    fullsize_1::Int = hi - lo
    d = sizeof(Int) * 8 - leading_zeros(fullsize_1)  # == ceil(Int, log(2, fullsize)

    if nthreads() > 2 && fullsize_1 + 1 ≥ PARALLEL_THRESHOLD
        # multithreading
        for p = 1:d, q = 1:p
            if p - q ≥ PARALLEL_THRESHOLD_D
                bsz = 1 << UInt(p - q + 1)
                # @threads for off = lo:bsz:hi-1, boff = 0:PARALLEL_THRESHOLD:((bsz>>0x01)-1)
                @threads for (off, boff) in [(off, boff) for boff = 0:PARALLEL_THRESHOLD:((bsz>>0x01)-1), off = lo:bsz:hi-1]
                    _sort_kernel!(x, off, off + PARALLEL_THRESHOLD - 1, p, q, o, boff)
                end
            else
                @threads for off = lo:PARALLEL_THRESHOLD:hi-1
                    _sort_kernel!(x, off, off + PARALLEL_THRESHOLD - 1, p, q, o)
                end
            end
        end
    else
        # single threading
        for p = 1:d, q = 1:p
            _sort_kernel!(x, lo, hi, p, q, o)
        end
    end
    return x
end

function _sort_kernel!(x::AbstractVector, lo, hi, p, q, o, boff=0)
    # @assert p ≥ q
    fullsize_1 = hi - lo
    d = 1 << UInt(p - q)
    sz_1 = min(d - 1, fullsize_1)
    for s = 0:2d:fullsize_1, ioff = boff:boff+sz_1
        joff = q == 1 ? (2d - ioff - 1) : ioff + d
        i = lo + s + ioff
        j = lo + s + joff
        if !lt(o, x[i], x[j])
            x[i], x[j] = x[j], x[i]
        end
    end
end

end # module

Main.MTBitonicSorter

In [4]:
using .BitonicSorter

In [5]:
using .MTBitonicSorter

In [6]:
x = [10, 30, 11, 20, 4, 330, 21, 110]

8-element Array{Int64,1}:
  10
  30
  11
  20
   4
 330
  21
 110

In [7]:
sort(x)

8-element Array{Int64,1}:
   4
  10
  11
  20
  21
  30
 110
 330

In [8]:
sort(x; alg=BitonicSort)

8-element Array{Int64,1}:
   4
  10
  11
  20
  21
  30
 110
 330

In [9]:
sort(x; alg=MTBitonicSort)

8-element Array{Int64,1}:
   4
  10
  11
  20
  21
  30
 110
 330

### Benchmark

In [10]:
using BenchmarkTools

In [11]:
@benchmark sort!(x) setup=(x=rand(Float64, 2^16))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.887 ms (0.00% GC)
  median time:      3.042 ms (0.00% GC)
  mean time:        3.051 ms (0.00% GC)
  maximum time:     4.414 ms (0.00% GC)
  --------------
  samples:          1607
  evals/sample:     1

In [12]:
@benchmark sort!(x; alg=BitonicSort) setup=(x=rand(Float64, 2^16))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     11.832 ms (0.00% GC)
  median time:      12.349 ms (0.00% GC)
  mean time:        12.380 ms (0.00% GC)
  maximum time:     14.479 ms (0.00% GC)
  --------------
  samples:          402
  evals/sample:     1

In [13]:
@benchmark sort!(x; alg=MTBitonicSort) setup=(x=rand(Float64, 2^16))

BenchmarkTools.Trial: 
  memory estimate:  412.19 KiB
  allocs estimate:  2872
  --------------
  minimum time:     3.348 ms (0.00% GC)
  median time:      3.459 ms (0.00% GC)
  mean time:        3.621 ms (0.68% GC)
  maximum time:     6.591 ms (20.88% GC)
  --------------
  samples:          1359
  evals/sample:     1

In [14]:
@benchmark sort!(x) setup=(x=rand(Float64, 2^24))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.193 s (0.00% GC)
  median time:      1.195 s (0.00% GC)
  mean time:        1.199 s (0.00% GC)
  maximum time:     1.215 s (0.00% GC)
  --------------
  samples:          4
  evals/sample:     1

In [15]:
@benchmark sort!(x; alg=BitonicSort) setup=(x=rand(Float64, 2^24))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     6.633 s (0.00% GC)
  median time:      6.633 s (0.00% GC)
  mean time:        6.633 s (0.00% GC)
  maximum time:     6.633 s (0.00% GC)
  --------------
  samples:          1
  evals/sample:     1

In [16]:
@benchmark sort!(x; alg=MTBitonicSort) setup=(x=rand(Float64, 2^24))

BenchmarkTools.Trial: 
  memory estimate:  3.34 MiB
  allocs estimate:  6903
  --------------
  minimum time:     2.384 s (0.00% GC)
  median time:      2.397 s (0.00% GC)
  mean time:        2.395 s (0.01% GC)
  maximum time:     2.405 s (0.02% GC)
  --------------
  samples:          3
  evals/sample:     1